In [1]:
import sys
print(sys.executable)
import os
import sys
if os.getcwd().endswith('visualize'):
    os.chdir('..')
sys.path.append(os.getcwd())


/usr/bin/python


In [2]:
import os
import sys
if os.getcwd().endswith('visualize'):
    os.chdir('..')
sys.path.append(os.getcwd())
from loader.dataloader import VideoDataSet,VideoDataLoader
import torch


dataset ="50salads"
split_test = "test.split2.bundle"
split_train = "train.split2.bundle"

default_path="./data/data/"
knowns = 14
unknowns = 5
prototypes = 30
video_dataset_test = VideoDataSet(dataset=dataset,
                               split=split_test,
                               default_path=default_path,
                               knowns=knowns,
                               unknowns=unknowns,
                               total_classes=knowns + prototypes)
data_loader_test = VideoDataLoader(video_dataset_test, batch_size=1, shuffle=True)

video_dataset_train = VideoDataSet(dataset=dataset,
                               split=split_train,
                               default_path=default_path,
                               knowns=knowns,
                               unknowns=unknowns,
                               total_classes=knowns + prototypes)
data_loader_train = VideoDataLoader(video_dataset_train, batch_size=1, shuffle=False)

tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ..., True, True, True])
tensor([True, True, True,  ...,

In [ ]:
from model.bert import ActionBERT, ActionBERTConfig
import torch.nn.functional as F
from model.boundary_model import ActionBoundary

knowns = 14
unknowns = 5
prototypes = 0
train_for_knowns = True

bert_conf = ActionBERTConfig(
    known_classes=knowns,
    input_dim=2048,
    d_model=256,
    num_heads=8,
    num_layers=3,
    local_window_size=128,
    window_dilation=32,
    dropout=0)
model = ActionBoundary(config=bert_conf, train_for_knowns=train_for_knowns)

path = "./output/ssad_first_try.pth"

state_dict = torch.load(path, map_location=torch.device('cuda'))
model.load_state_dict(state_dict, strict=False)
model = model.to('cuda')
model.eval()
print("Modell erfolgreich geladen.")

def get_model_pred(features,padding_mask):
    unknown_mask = torch.zeros_like(padding_mask).bool()
    res  = model(features,padding_mask)
       
    return res
 

Modell erfolgreich geladen.


In [7]:
import matplotlib.pyplot as plt
import numpy as np
import torch


for batch_test in data_loader_train:
    features = batch_test["features"].to('cuda')
    unknown_mask = batch_test["unknown_mask"] .to('cuda')
    target_truth = batch_test["target_truth"].to('cuda')
    padding_mask = batch_test["padding_mask"].to('cuda')
    labels_dict = batch_test["labels_dict"]
    break


import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import torch

def visualize_model_votings_and_gt(batch, model_result, idx=0):
    """
    Wandelt die Modell-Vorhersage (Progress) in Votings (Peaks) um 
    und vergleicht sie mit der Ground Truth.
    """
    
    # --- 1. Daten extrahieren ---
    padding_mask = batch["padding_mask"][idx].cpu()
    valid_len = padding_mask.sum().int().item()
    
    target_truth = batch["target_truth"][idx].cpu().numpy()[:valid_len]
    unknown_mask = batch["unknown_mask"][idx].cpu().numpy()[:valid_len].astype(bool)
    
    # Modell Vorhersage holen (Progress Kanal 0)
    if "action_progress" in model_result:
        pred_prog = model_result["action_progress"][idx, 0].detach().cpu().numpy()[:valid_len]
    elif "boundary_pred" in model_result:
        pred_prog = model_result["boundary_pred"][idx, 0].detach().cpu().numpy()[:valid_len]
    else:
        print("Error: Kein Progress-Signal im Output gefunden.")
        return

    # --- 2. Votings Berechnen (Vom Rampen-Signal zu Peaks) ---
    # Wir suchen den "Absturz" des Signals. 
    # Wenn t=0.9 und t+1=0.1, ist die Differenz +0.8. Das ist ein Peak für eine Grenze.
    # Wenn t=0.5 und t+1=0.51, ist die Differenz -0.01. Das ist kein Peak.
    
    # Berechnung: diff = current - next
    # Wir hängen eine 0 an, damit die Länge gleich bleibt
    prog_padded = np.append(pred_prog, 0)
    voting_signal = prog_padded[:-1] - prog_padded[1:]
    
    # ReLU: Wir interessieren uns nur für positive Differenzen (Abstürze)
    # Alles was negativ ist (normaler Anstieg der Rampe), setzen wir auf 0.
    voting_signal = np.maximum(voting_signal, 0)
    
    # Optional: Signal etwas verstärken für bessere Sichtbarkeit
    voting_signal = voting_signal * 1.5

    # --- 3. Farben vorbereiten ---
    cmap = plt.get_cmap('tab20')
    colors = cmap.colors
    
    NUM_KNOWNS = 14 # Deine Config
    unique_ids = np.unique(target_truth)
    id_to_color = {}
    
    for uid in unique_ids:
        if uid >= NUM_KNOWNS: 
            # Unknowns bekommen Dunkelgrau (sichtbar, aber unterscheidbar von Bunt)
            id_to_color[uid] = np.array([0.3, 0.3, 0.3]) 
        else:
            id_to_color[uid] = colors[uid % len(colors)]

    # GT Balken bild
    gt_bar = np.zeros((1, valid_len, 3))
    for t in range(valid_len):
        gt_bar[0, t] = id_to_color[target_truth[t]]

    # --- 4. Plotten ---
    fig, (ax_vote, ax_gt) = plt.subplots(2, 1, figsize=(16, 8), sharex=True, 
                                         gridspec_kw={'height_ratios': [3, 1]})
    
    # PLOT 1: Model Votings (Peaks)
    x_axis = range(valid_len)
    ax_vote.plot(x_axis, voting_signal, color='blue', linewidth=1.5, label='Model Boundary Votes')
    # Fülle die Fläche unter der Kurve
    ax_vote.fill_between(x_axis, 0, voting_signal, color='blue', alpha=0.2)
    
    # Highlight: Unknown Regionen (Hintergrund rot schattiert)
    ax_vote.fill_between(x_axis, 0, voting_signal.max() + 0.1, where=unknown_mask, 
                         color='red', alpha=0.1, label='Unknown Region (Target)')
    
    # Hilfslinien: Echte GT Grenzen einzeichnen
    gt_boundaries = np.where(np.diff(target_truth) != 0)[0]
    for b in gt_boundaries:
        ax_vote.axvline(x=b, color='green', linestyle='--', alpha=0.5, linewidth=1)

    ax_vote.set_title("Model Predicted Boundaries (Blue Peaks)", fontweight='bold')
    ax_vote.set_ylabel("Voting Strength")
    ax_vote.legend(loc='upper right')
    ax_vote.grid(True, alpha=0.3)

    # PLOT 2: Ground Truth
    ax_gt.imshow(gt_bar, aspect='auto', interpolation='nearest')
    ax_gt.set_title("Ground Truth (Grey = Unknown Actions)", fontweight='bold')
    ax_gt.set_yticks([])
    ax_gt.set_xlabel("Frames")

    # Legende für GT
    patches = [mpatches.Patch(color=[0.3, 0.3, 0.3], label="Unknown Action")]
    # Ein paar Knowns beispielhaft
    for i in range(min(3, len(unique_ids))):
        uid = unique_ids[i]
        if uid < NUM_KNOWNS:
            patches.append(mpatches.Patch(color=id_to_color[uid], label=f"Known {uid}"))
            
    ax_gt.legend(handles=patches, bbox_to_anchor=(1.01, 1), loc='upper left')

    plt.tight_layout()
    plt.show()

# --- ANWENDUNG ---
# 1. Forward Pass
model.eval()
with torch.no_grad():
    features = batch_test["features"].to('cuda')
    padding_mask = torch.ones_like(padding_mask).bool() 
    # Falls dein Forward Patch Mask braucht:
    patch_mask = torch.ones_like(padding_mask).bool() 
    
    result = get_model_pred(features, padding_mask)

# 2. Visualisieren
visualize_model_votings_and_gt(batch_test, result, idx=0)


Exception: NaN values detected in GLOBAL attention output